# Swaps de Tasa de Interés

## Configuración

### Librerías

In [1]:
from finrisk import QC_Financial_3 as Qcf
from IPython.display import Image
from IPython.core.display import HTML
import pandas as pd
from datetime import date

### Variables Globales

In [2]:
bus_cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)

In [3]:
format_dict = {'nominal': '{0:,.2f}', 'amort': '{0:,.2f}', 'interes': '{0:,.2f}', 'flujo': '{0:,.2f}',
               'amortizacion': '{0:,.2f}',
               'icp_inicial': '{0:,.2f}', 'icp_final': '{0:,.2f}',
               'valor_tasa': '{0:,.4%}', 'spread': '{0:,.4%}', 'gearing': '{0:,.2f}',
               'amort_moneda_pago': '{0:,.2f}', 'interes_moneda_pago': '{0:,.2f}', 'valor_indice_fx': '{0:,.2f}'}

## Definición de un Swap de Tasa de Interés

- Un swap de tasa de interés es un contrato a través del cual dos partes se comprometen a intercambiar flujos de intereses calculados a una tasa fija por intereses calculados a una tasa flotante (algún referencial válido y ampliamente aceptado por el mercado).


- Por ejemplo, cada 3M y por un período de 2Y se intercambian flujos calculados al 0,2181% versus flujos calculados utilizando la Libor USD 3M.
  - Los flujos se deben calcular sobre un cierto nominal (nocional) que nunca se intercambia.

Para más sobre la [Libor](https://www.theice.com/iba/libor), en particular los párrafos bajo el encabezado **The Future of Libor**. 

**Pantalla de cotización en Bloomberg.**

In [4]:
Image(url="img/libor_usd_3m_fixed_2y_swpm.gif", width=900, height=720)

Notar las diferencias entre ambas patas,

- **periodicidad:** en la pata fija (Recibir) la periodicidad es semestral, mientras que en la pata flotante (Pagar) la periodicidad es trimestral.
- **Días (fracción de año):** en la pata fija (Recibir) la fracción de año es 30/360, mientras que en la pata flotante (Pagar) es Act/360.

**Flujos de Caja (la pata flotante es estimada)**

In [5]:
Image(url="img/libor_usd_3m_fixed_2y_cashflows.gif", width=900, height=720)

**¿Cómo construir estos flujos con la librería?**

Partamos con la pata fija.

In [6]:
# Recibo o pago los flujos de esta pata
rp = Qcf.RecPay.RECEIVE

# Fecha de inicio de devengo del primer cupón
fecha_inicio = Qcf.QCDate(17, 9, 2020)

# Fecha final de devengo, antes de ajustes, del último cupón
fecha_final = Qcf.QCDate(17, 9, 2022)

# Regla para ajustes de días feriados
bus_adj_rule = Qcf.BusyAdjRules.MODFOLLOW

# Periodicidad de pago
periodicidad = Qcf.Tenor('6M')

# Tipo de período irregular (si lo hay)
periodo_irregular = Qcf.StubPeriod.SHORTFRONT

# Número de días después de la fecha final de devengo en que se paga el flujo
lag_pago = 0

# Nocional del contrato
nocional = 10000000.0
amort_es_flujo = True
valor_tasa = .002181
tasa_cupon = Qcf.QCInterestRate(valor_tasa, Qcf.QC30360(), Qcf.QCLinearWf())
moneda = Qcf.QCUSD()
es_bono = False

# Se da de alta el objeto
fixed_rate_leg = Qcf.LegFactory.build_bullet_fixed_rate_leg(rp,
                                                            fecha_inicio,
                                                            fecha_final,
                                                            bus_adj_rule,
                                                            periodicidad,
                                                            periodo_irregular,
                                                            bus_cal,
                                                            lag_pago,
                                                            nocional,
                                                            amort_es_flujo,
                                                            tasa_cupon,
                                                            moneda,
                                                            es_bono)

In [7]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, fixed_rate_leg.size()):
    tabla.append(Qcf.show(fixed_rate_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_ini', 'fecha_fin', 'fecha_pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo', 'moneda',
            'valor_tasa', 'tipo_tasa']
df = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df.style.format(format_dict)

,fecha_ini,fecha_fin,fecha_pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2020-09-17,2021-03-17,2021-03-17,"10,000,000.00",0.00,"10,905.00",True,"10,905.00",USD,0.2181%,Lin30360
1,2021-03-17,2021-09-17,2021-09-17,"10,000,000.00",0.00,"10,905.00",True,"10,905.00",USD,0.2181%,Lin30360
2,2021-09-17,2022-03-17,2022-03-17,"10,000,000.00",0.00,"10,905.00",True,"10,905.00",USD,0.2181%,Lin30360
3,2022-03-17,2022-09-19,2022-09-19,"10,000,000.00","10,000,000.00","11,026.17",True,"10,011,026.17",USD,0.2181%,Lin30360


Pata Flotante

In [9]:
# Se da de alta los parámetros requeridos
rp = Qcf.RecPay.PAY

periodicidad_pago = Qcf.Tenor('3M')
periodo_irregular_pago = Qcf.StubPeriod.NO
lag_pago = 0
periodicidad_fijacion = Qcf.Tenor('3M')
periodo_irregular_fijacion = Qcf.StubPeriod.SHORTFRONT

# vamos a usar el mismo calendario para pago y fijaciones
lag_de_fijacion = 2

# Definición del índice
codigo = 'LIBORUSD3M'
lin_act360 = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('2d')
tenor = Qcf.Tenor('3m')
usd = Qcf.QCUSD()
libor_usd_3m = Qcf.InterestRateIndex(codigo,
                                     lin_act360,
                                     fixing_lag,
                                     tenor,
                                     bus_cal,
                                     bus_cal,
                                     usd)
# Fin índice

spread = .0
gearing = 1.0

ibor_leg = Qcf.LegFactory.build_bullet_ibor2_leg(rp, fecha_inicio, fecha_final, bus_adj_rule, periodicidad_pago,
                                                 periodo_irregular_pago, bus_cal, lag_pago,
                                                 periodicidad_fijacion, periodo_irregular_fijacion,
                                                 bus_cal, lag_de_fijacion, libor_usd_3m,
                                                 nocional, amort_es_flujo, moneda, spread, gearing)

In [10]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, ibor_leg.size()):
    tabla.append(Qcf.show(ibor_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = Qcf.get_column_names('IborCashflow', '')
df = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df.style.format(format_dict)

,fecha_inicial,fecha_final,fecha_fixing,fecha_pago,nominal,amortizacion,interes,amort_es_flujo,flujo,moneda,codigo_indice_tasa,valor_tasa,spread,gearing,tipo_tasa
0,2020-09-17,2020-12-17,2020-09-15,2020-12-17,"-10,000,000.00",0.00,-0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
1,2020-12-17,2021-03-17,2020-12-15,2021-03-17,"-10,000,000.00",0.00,-0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
2,2021-03-17,2021-06-17,2021-03-15,2021-06-17,"-10,000,000.00",0.00,-0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
3,2021-06-17,2021-09-17,2021-06-15,2021-09-17,"-10,000,000.00",0.00,-0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
4,2021-09-17,2021-12-17,2021-09-15,2021-12-17,"-10,000,000.00",0.00,-0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
5,2021-12-17,2022-03-17,2021-12-15,2022-03-17,"-10,000,000.00",0.00,-0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
6,2022-03-17,2022-06-17,2022-03-15,2022-06-17,"-10,000,000.00",0.00,-0.00,True,0.00,USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360
7,2022-06-17,2022-09-19,2022-06-15,2022-09-19,"-10,000,000.00","-10,000,000.00",-0.00,True,"-10,000,000.00",USD,LIBORUSD3M,0.0000%,0.0000%,1.00,LinAct360


**¿Para qué sirven?**

- Empresas que tengan deuda en tasa flotante y quieran protegerse contra subidas de tasas de interés.


- Inversionistas con activos a tasa fija y quieran protegerse contra subidas de tasa de interés.


- Empresas con deuda a tasa fija que quieran protegerse contra caídas de las tasas de interés.


- Toma de posiciones direccionales en tasa de interés (al alza y a la baja).

**Otros tipos de swaps**

- Flotante vs Flotante en la misma divisa: por ejemplo un swap en el cual se intercambian flujos calculados a la Libor USD de 3M por flujos calculados a la Libor USD de 1Y.


- Fijo vs Fijo en dos divisas: por ejemplo un swap en el que se intercambian flujos calculados sobre un nocional de $M_{USD}$ USD usando una tasa fija de $r_{USD}$ por flujos calculados sobre un nocional de $M_{EUR}$ EUR usando una tasa fija de $r_{EUR}$ (los swaps con dos divisas se llaman Cross Currency Swaps (CCS)).


- Fijo vs Variable en dos divisas: por ejemplo un swap en el que se intercambian flujos calculados sobre un nocional de $M_{USD}$ USD usando una tasa fija de $r_{USD}$ por flujos calculados sobre un nocional de $M_{EUR}$ EUR usando Libor EUR de 3M.


- Variable vs Variable en dos divisas: por ejemplo un swap en el que se intercambian flujos calculados sobre un nocional de $M_{USD}$ USD usando una Libor USD de 3M por flujos calculados sobre un nocional de $M_{EUR}$ EUR usando Libor EUR de 3M.

## Overnight Index Swaps (OIS)

- Los OIS son swaps en los cuales se intercambian flujos calculados a una tasa fija por flujos calculados a partir de una tasa overnight interbancaria componiendo intereses.


- Por ejemplo, en USD se utiliza la effective federal funds rate, en EUR el llamado EONIA y en GBP el índice SONIA.


- La effective federal funds rate corresponde al promedio de las transacciones de financiamiento no colateralizado informado a la Reserva Federal de Nueva York (lineal Act/360).


- El índice EONIA (European Overnight Index Average) se computa como el promedio de las transacciones overnight de un grupo pre establecido de bancos (lineal Act/360).


- El índice SONIA (Sterling Overnight Index Average) se calcula como el promedio de las transacciones overnight en Londres ejecutadas a través de WMBA (The Wholesale Markets Brokers’ Association). Este índice es lineal Act/365.


- El mercado USD está transitando hacia el uso de un nuevo benchmark overnight, la Secured Overnight Financing Rate o **SOFR**. Ver [SOFR Primer](https://www.sifma.org/wp-content/uploads/2019/07/SIFMA-Insights-SOFR-Primer.pdf).

### Flujos de un OIS

- En un OIS, los flujos de la pata flotante se obtienen de la composición de los factores de capitalización asociados al índice overnight.


- Por ejemplo, suponiendo que para la pata flotante de un OIS se utiliza la effective fed funds rate de agosto 2014:

In [11]:
# Se importa el archivo Excel a un DataFrame de pandas
eff_1 = pd.read_excel('data/effective_fed_funds_1.xlsx')

# Los campos de fecha se importan como datetime (con horas, minutos y segundos).
# Se mantiene sólo la fecha.
eff_1['fecha_inicial'] = eff_1['fecha_inicial'].dt.date
eff_1['fecha_final'] = eff_1['fecha_final'].dt.date

# Se visualiza el resultado
eff_1.style.format({'effective_fed_funds': '{0:.2%}'})

,fecha_inicial,fecha_final,effective_fed_funds
0,2014-08-01,2014-08-04,0.09%
1,2014-08-04,2014-08-05,0.09%
2,2014-08-05,2014-08-06,0.09%
3,2014-08-06,2014-08-07,0.09%
4,2014-08-07,2014-08-08,0.09%
5,2014-08-08,2014-08-11,0.09%
6,2014-08-11,2014-08-12,0.09%
7,2014-08-12,2014-08-13,0.09%
8,2014-08-13,2014-08-14,0.09%
9,2014-08-14,2014-08-15,0.09%


Primero se calcula el factor de capitalización entre las fechas iniciales y finales de cada una de las tasas. Se define una pequeña función auxiliar:

In [12]:
def get_wf(fecha_inicial: date, fecha_final: date, valor_tasa: float) -> float:
    """
    Retorna el factor de capitalización para una tasa (cuyo valor es uno de los argumentos
    de la función) entre dos fechas. Supone que la convención de la tasa es LinAct360.
    """
    qc_fec_ini = Qcf.QCDate(fecha_inicial.day, fecha_inicial.month, fecha_inicial.year)
    qc_fec_fin = Qcf.QCDate(fecha_final.day, fecha_final.month, fecha_final.year)
    qc_tasa = Qcf.QCInterestRate(valor_tasa, Qcf.QCAct360(), Qcf.QCLinearWf())
    return qc_tasa.wf(qc_fec_ini, qc_fec_fin)

Se aplica esta función para agregar la columna **wf** al `DataFrame` anterior.

In [13]:
eff_1['wf'] = eff_1.apply(
    lambda row: get_wf(row['fecha_inicial'], row['fecha_final'], row['effective_fed_funds']),
    axis=1
)

# Se visualiza el resultado
eff_1.style.format({'effective_fed_funds': '{0:.2%}', 'wf': '{0:.6%}'})

,fecha_inicial,fecha_final,effective_fed_funds,wf
0,2014-08-01,2014-08-04,0.09%,100.000750%
1,2014-08-04,2014-08-05,0.09%,100.000250%
2,2014-08-05,2014-08-06,0.09%,100.000250%
3,2014-08-06,2014-08-07,0.09%,100.000250%
4,2014-08-07,2014-08-08,0.09%,100.000250%
5,2014-08-08,2014-08-11,0.09%,100.000750%
6,2014-08-11,2014-08-12,0.09%,100.000250%
7,2014-08-12,2014-08-13,0.09%,100.000250%
8,2014-08-13,2014-08-14,0.09%,100.000250%
9,2014-08-14,2014-08-15,0.09%,100.000250%


Se calcula el producto de todos los factores de capitalización y luego se deduce la tasa equivalente en convención LinAct360 entre la primera fecha inicial y la última fecha final.

In [14]:
prod = eff_1['wf'].product()

qc_fec_ini = Qcf.QCDate(
    eff_1.iloc[0]['fecha_inicial'].day,
    eff_1.iloc[0]['fecha_inicial'].month,
    eff_1.iloc[0]['fecha_inicial'].year
)
qc_fec_fin = Qcf.QCDate(
    eff_1.iloc[20]['fecha_final'].day,
    eff_1.iloc[20]['fecha_final'].month,
    eff_1.iloc[20]['fecha_final'].year
)

dias = qc_fec_ini.day_diff(qc_fec_fin)
tasa_eq = (prod - 1) * 360.0 / dias

print(f'Producto wf: {prod:.6%}')
print(f'Días totales: {dias}')
print(f'Tasa equivalente: {tasa_eq:.6%}')

Producto wf: 100.007584%
Días totales: 31
Tasa equivalente: 0.088068%


Finalmente, para obtener los intereses de la pata flotante, se aplica la tasa equivalente, por el número de días del cupón, al nocional del contrato.

In [15]:
nocional = 100000000
interes_flot_ois = nocional * tasa_eq * dias / 360
print(f"Para un nocional de {nocional:,.0f} los intereses de la pata flotante del OIS son: {interes_flot_ois:,.2f} .")

Para un nocional de 100,000,000 los intereses de la pata flotante del OIS son: 7,583.60 .


Utilizando la librería:

In [16]:
qc_tasa_eq = Qcf.QCInterestRate(tasa_eq, Qcf.QCAct360(), Qcf.QCLinearWf())
print(f'Intereses: {nocional * (qc_tasa_eq.wf(qc_fec_ini, qc_fec_fin) - 1):,.2f}')

Intereses: 7,583.60


- Se cotizan distintas madureces (con distinta liquidez) hasta 30Y.


- Los tenors de corto plazo son contratos que tienen una única compensación al vencimiento.


- Los tenors desde 2Y en adelante suelen tener cupones intermedios, siendo las periodicidades más comunes 1M, 3M, 6M y 1Y.


- En particular, dado que el índice flotante es overnight, este parámetro es totalmente configurable en un OIS.


- Esto no es así en un swap de Libor u otro índice similar, en donde nos vemos restringidos por el plazo de la tasa flotante.

## Swaps del Mercado Chileno

- En el mercado interbancario chileno, uno de los swaps que se opera regularmente y para el cual es posible encontrar precios de mercado a distintos plazos es el **swap cámara promedio en CLP**.


- El comportamiento de la tasa flotante replica el costo de financiamiento (beneficio de inversión) que se tiene tomando (pasando) fondos en forma diaria a la tasa interbancaria promedio a 1 día componiendo intereses.


- Dicho de otra manera, un swap cámara promedio en CLP, es un OIS, cuyo índice overnight es la tasa interbancaria promedio a 1 día.

- Los plazos líquidos que los bancos cotizan son 3M, 6M, 9M, 1Y, 1Y6M, entre 2Y y 10Y plazos anuales, 12Y, 15Y, 20Y


- Así como los OIS en otras divisas, los cinco plazos más cortos son swaps sin cupones, es decir hay una única compensación de intereses al vencimiento del swap.


- Los swaps de plazos más largos tienen pagos semestrales en ambas patas.

In [17]:
Image(url="img/20200917_swap_icp.png", width=450*1.2, height=360*1.2)

**Fuente:** Closing de ICAP al 17-09-2020.

In [18]:
Qcf.get_column_names('IcpClpCashflow', '')

('fecha_inicial',
 'fecha_final',
 'fecha_pago',
 'nominal',
 'amortizacion',
 'amort_es_flujo',
 'flujo',
 'moneda',
 'icp_inicial',
 'icp_final',
 'valor_tasa',
 'interes',
 'spread',
 'gearing',
 'tipo_tasa')